# Partie 3 - Étude des Déplacements de Réponses LLM par Rapport à un Référentiel Neutre


## Introduction et objectif
Ce notebook constitue le prolongement méthodologique de notre première étude (notebook Partie 2) sur les matrices de transition *ton question - valence réponse* des grands modèles de langage (LLM). Alors que notre travail initial a identifié des patterns significatifs dans la façon dont les LLMs répondent à des questions formulées avec différentes tonalités, cette nouvelle analyse vise à **quantifier avec précision l'influence spécifique du ton sur les réponses**.



**Objectifs spécifiques**:
* Mesurer les déplacements des réponses par rapport à un **référentiel neutre** pour isoler l'effet du ton
* Quantifier l'amplitude et la direction des déviations induites par les tonalités positive et négative
* Comparer ces déplacements entre **sujets banals** et **sujets sensibles/controversés**
* Vérifier si il y a une relation causale entre la tonalité d'une question et la déviation de la réponse par rapport à la position neutre.

Cette approche *méthodologique* robuste nous permettra de distinguer l'influence du ton de celle du contenu substantiel de la question, en utilisant systématiquement la réponse à la formulation neutre comme point de référence pour chaque question.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import networkx as nx
from tqdm.notebook import tqdm

# Pour une meilleure visualisation
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Définir la graine aléatoire pour la reproductibilité
np.random.seed(42)

In [ ]:
# Paramètres globaux
VERBOSE = True
SEUIL_CONFIANCE = 0.7  # Seuil pour considérer une réponse comme "confiante"

## 📦 Bloc 1 – Configuration et chargement des données

Dans cette première partie, nous allons charger les données générées lors de notre première expérience. Pour rappel, ces données contiennent les questions posées à ChatGPT avec différents tons, ainsi que les réponses obtenues et leur classification.

Le fichier attendu est `output-LLM-responses-v3.csv`, généré lors de la partie 1 de notre analyse. Ce fichier contient plusieurs colonnes importantes :
- Le `domaine` de la question (science, société, santé, etc.)
- `question` : le texte de la question posée
- `ton` : le ton utilisé pour poser la question
- `reponse` : la réponse fournie par ChatGPT
- `label` : la classification de la réponse, le `label` attribué à la réponse (`positif, négatif, neutre`)
- `confidence` : le niveau de confiance de la classification


In [ ]:
# Chargement des données
try:
    df = pd.read_csv("output-LLM-responses-v3.csv" , sep=',')
    # Suppression des colonnes vides ou inutiles
    df = df.drop(columns=['tonalité', 'Unnamed: 6'], errors='ignore')
    print(f"Données chargées : {df.shape[0]} questions analysées")
except FileNotFoundError:
    print("❌ Fichier de données non trouvé. Vérifiez que le notebook partie 1 a bien été exécuté.")

In [ ]:
df.info()

In [ ]:
df.head(10)

In [ ]:
# visualiser un exemple
import random
for bcl in range(5):
  idx = random.randint(0 , df.shape[0])
  print('Question ', idx , ' : ', 'ton '  , df.iloc[idx]['ton'], ' : ' ,df.iloc[idx]['question'])

In [ ]:
df.iloc[30:40]

##Organisation des triplets de questions
Dans cette étude, nous analysons l'influence du ton sur les réponses du LLM en utilisant des triplets de questions.
Chaque triplet contient la même question formulée avec trois tonalités différentes (`positive, neutre, négative`). Pour faciliter cette analyse, nous ajoutons un identifiant de groupe (`group_id`) à notre jeu de données, permettant d'identifier rapidement les trois variantes tonales d'une même question substantielle.

Cette structure nous permet d'utiliser systématiquement la **version neutre comme référence** pour mesurer les déplacements induits par les tonalités positive et négative.

In [ ]:
# id de groupe pour repérer chq triplet ....
df['group_id'] = df.index // 3

In [ ]:
df.tail(3)

In [ ]:
# Vérification que chaque groupe contient bien les 3 tonalités
group_counts = df.groupby('group_id')['ton'].nunique()
if (group_counts != 3).any():
    print("⚠️ Attention: Certains groupes n'ont pas exactement 3 tonalités!")
else:
    print("✅ Tous les groupes contiennent les 3 tonalités (positif, neutre, négatif)")

In [ ]:
group_counts

In [ ]:
# Créer un DataFrame réorganisé, avec une ligne par groupe et les informations des 3 questions/réponses
triplets = []

for group_id, group_df in df.groupby('group_id'):
    # Récupérer les données par tonalité
    positif = group_df[group_df['ton'] == 'positif'].iloc[0] if not group_df[group_df['ton'] == 'positif'].empty else None
    neutre = group_df[group_df['ton'] == 'neutre'].iloc[0] if not group_df[group_df['ton'] == 'neutre'].empty else None
    negatif = group_df[group_df['ton'] == 'négatif'].iloc[0] if not group_df[group_df['ton'] == 'négatif'].empty else None

    # Vérifier que nous avons bien les 3 tonalités
    if positif is not None and neutre is not None and negatif is not None:
        triplet = {
            'group_id': group_id,
            'domaine': neutre['domaine'],
            'question_base': neutre['question'],  # Utiliser la question neutre comme base

            # Questions
            'question_positif': positif['question'],
            'question_neutre': neutre['question'],
            'question_negatif': negatif['question'],

            # Réponses
            'reponse_positif': positif['reponse'],
            'reponse_neutre': neutre['reponse'],
            'reponse_negatif': negatif['reponse'],

            # Labels
            'label_positif': positif['label'],
            'label_neutre': neutre['label'],
            'label_negatif': negatif['label'],

            # Confiance
            'conf_positif': positif['confidence'],
            'conf_neutre': neutre['confidence'],
            'conf_negatif': negatif['confidence'],

            # Sensibilité (si présente)
            'sensible': neutre.get('sensible', None)
        }
        triplets.append(triplet)


In [ ]:
triplets

In [ ]:
# Conversion en dataframe
triplets_df = pd.DataFrame(triplets)

In [ ]:
triplets_df['domaine'].unique()

In [ ]:


# Ajout d'une colonne pour le type de sujet (sensible ou neutre)
if False:#'sensible' in triplets_df.columns:
    triplets_df['type_sujet'] = triplets_df['sensible'].apply(lambda x: 'Sensible' if x == 'oui' else 'Neutre')
else:
    # Si l'information n'est pas disponible, on peut l'inférer à partir du domaine
    sensible_domains = ['science', 'opinion', 'société', 'santé', 'technologie',
       'éducation', 'politique', 'santé publique', 'justice', 'sécurité',
       'genre', 'immigration', 'climat']
    triplets_df['type_sujet'] = triplets_df['domaine'].apply(
        lambda x: 'sensible' if x.lower() in sensible_domains else 'banal'
    )

# Affichage (random) pour vérification
print(f"Nombre de triplets: {len(triplets_df)}")
triplets_df.sample(8)#[['group_id', 'domaine', 'question_base', 'label_positif', 'label_neutre', 'label_negatif']].head(3)

## Matrices de transition avec référentiel neutre

### Méthode pour la matrice de transition:

Pour analyser l'influence de la tonalité sur les réponses, nous allons créer une matrice de transition basée sur le référentiel neutre:

Nous prendrons chaque triplet et comparerons les labels des tonalités positive et négative par rapport au label neutre

La matrice de transition montrera:

* Combien de fois une question neutre avec label X a donné un label Y avec une tonalité positive (changement de label : X -> Y)
* Combien de fois une question neutre avec label X a donné un label Z avec une tonalité négative.

Cela nous permettra de visualiser si la tonalité provoque un changement de position (par exemple, `neutre → positif` ou `neutre → négatif`) et de quantifier ces "déplacements" d'opinion.

Cette approche permettra de voir si certaines tonalités influencent le modèle à se déplacer vers une position plus positive ou plus négative.

In [ ]:
# Création de matrices de transition pour analyser le déplacement depuis la réponse neutre
# Nous créons deux matrices: une pour la tonalité positive et une pour la négative

# Identifions d'abord tous les labels possibles pour créer nos matrices
unique_labels = sorted(df['label'].unique())

# Matrice pour la transition neutre → positif
matrix_neutre_positif = pd.DataFrame(0, index=unique_labels, columns=unique_labels)

# Matrice pour la transition neutre → négatif
matrix_neutre_negatif = pd.DataFrame(0, index=unique_labels, columns=unique_labels)

# Parcourir chaque groupe pour remplir les matrices
for group_id, group in df.groupby('group_id'):
    # Récupérer les réponses pour chaque tonalité
    neutre_row = group[group['ton'] == 'neutre']
    positif_row = group[group['ton'] == 'positif']
    negatif_row = group[group['ton'] == 'négatif']

    # Vérifier que nous avons bien les trois tonalités
    if len(neutre_row) == 1 and len(positif_row) == 1 and len(negatif_row) == 1:
        # Récupérer les labels
        label_neutre = neutre_row.iloc[0]['label']
        label_positif = positif_row.iloc[0]['label']
        label_negatif = negatif_row.iloc[0]['label']

        # Mettre à jour les matrices
        matrix_neutre_positif.loc[label_neutre, label_positif] += 1
        matrix_neutre_negatif.loc[label_neutre, label_negatif] += 1

In [ ]:
matrix_neutre_positif

### Interprétation de la matrice de transition (`neutre_positif`):

Cette matrice montre comment les réponses changent lorsqu'on passe d'une question neutre à une question avec **tonalité positive**.

#### Exemple:
* `'question_neutre'`: `'Changer ses draps chaque semaine réduit-il les allergies'`

La réponse du LLM :

*  "`Oui, changer régulièrement ses draps peut aider à réduire les allergies. Les draps peuvent accumuler des allergènes tels que les acariens, les squames d'animaux et le pollen. En les changeant chaque semaine, on réduit l'exposition à ces allergènes, ce qui peut atténuer les symptômes allergiques.`",

Le label de la réponse (ton neutre) : label : `positif`'.

Maintenant, **changement de ton** pour la question posée. Le ton devient '`positif`'
* `"C'est évident que changer ses draps chaque semaine réduit les allergies, non ?"`,

La réponse du LLM :

* "`C'est en partie vrai. Changer ses draps chaque semaine peut aider à réduire les allergènes tels que les acariens. Cependant, d'autres facteurs entrent également en jeu dans la réduction des allergies, comme la propreté de l'ensemble de la maison et l'évitement des allergènes spécifiques. Donc, bien que cela puisse aider, changer les draps chaque semaine n'est pas une solution complète.`"

Le label de réponse (ton positif) : label : '`neutre`'

**Interprétation** : le changement de ton (neutre -> positif) provoque un changement de label (valence) de la réponse (positif -> neutre)





**Lecture**: Chaque ligne représente le label de la réponse neutre, et chaque colonne représente le label de la réponse avec tonalité positive:
label question (ligne) -> label réponse (colonne)


* Diagonale principale (18, 5, 17): Ces nombres indiquent les cas où la tonalité positive n'a pas changé le label de la réponse par rapport à la tonalité neutre. Par exemple, 18 questions qui avaient une réponse neutre avec la formulation neutre ont gardé une réponse neutre même avec la formulation positive.

* 7 (ligne positif, colonne neutre): Ceci est particulièrement intéressant - dans 7 cas, une question qui avait reçu une réponse positive avec une formulation neutre a obtenu une réponse neutre lorsqu'elle a été posée avec une tonalité positive. Cela suggère un effet modérateur inattendu de la tonalité positive.

* 3 (ligne neutre, colonne négatif): Dans 3 cas, une question avec réponse neutre est devenue négative lorsque formulée positivement, ce qui est contre-intuitif.

* 1 (ligne négatif, colonne neutre): Un cas où une question avec réponse négative est devenue neutre quand formulée positivement, ce qui est cohérent avec l'idée que la tonalité positive peut atténuer les positions négatives.

En résumé, cette matrice montre que l'effet de la tonalité positive est complexe - elle ne pousse pas systématiquement les réponses vers un label plus positif, et dans certains cas, elle semble même avoir un effet modérateur sur des questions qui auraient autrement reçu une réponse positive.

Pour extraire les **exemples** illustrant ce cas particulier où une question avec réponse positive en tonalité neutre devient neutre en tonalité positive

In [ ]:
# Extraction des exemples où question neutre → réponse positive mais question positive → réponse neutre
exemples = []

for group_id, group in df.groupby('group_id'):
    # Récupérer les lignes pour chaque tonalité
    neutre_row = group[group['ton'] == 'neutre']
    positif_row = group[group['ton'] == 'positif']

    # Vérifier si nous avons le cas recherché
    if (len(neutre_row) == 1 and len(positif_row) == 1 and
        neutre_row.iloc[0]['label'] == 'positif' and
        positif_row.iloc[0]['label'] == 'neutre'):

        # Déterminer le nom de la colonne contenant la réponse
        # Cherchons les colonnes possibles
        resp_cols = [col for col in df.columns if 'resp' in col.lower()]
        resp_col = resp_cols[0] if resp_cols else 'reponse'  # Utiliser le premier trouvé ou 'reponse' par défaut

        exemple = {
            'group_id': group_id,
            'question_neutre': neutre_row.iloc[0]['question'],
            'reponse_neutre': neutre_row.iloc[0][resp_col],
            'label_neutre': neutre_row.iloc[0]['label'],
            'question_positive': positif_row.iloc[0]['question'],
            'reponse_positive': positif_row.iloc[0][resp_col],
            'label_positive': positif_row.iloc[0]['label']
        }
        exemples.append(exemple)

# Afficher exemples
for i, exemple in enumerate(exemples[:]):
    print(f"\nExemple {i+1}:")
    print(f"Question neutre: {exemple['question_neutre']}")
    print(f"Réponse neutre (label: {exemple['label_neutre']}): {exemple['reponse_neutre'][:200]}...")
    print(f"Question positive: {exemple['question_positive']}")
    print(f"Réponse positive (label: {exemple['label_positive']}): {exemple['reponse_positive'][:200]}...")

In [ ]:
matrix_neutre_negatif

### Interprétation de la matrix `netre_negatif`
**Cette matrice montre comment les réponses changent lorsqu'on passe d'une question neutre à une question avec tonalité négative.**

#### Exemple:
* `'question_neutre'`: `'Changer ses draps chaque semaine réduit-il les allergies'`

La réponse du LLM :

*  '`reponse_neutre`': "`Oui, changer régulièrement ses draps peut aider à réduire les allergies. Les draps peuvent accumuler des allergènes tels que les acariens, les squames d'animaux et le pollen. En les changeant chaque semaine, on réduit l'exposition à ces allergènes, ce qui peut atténuer les symptômes allergiques.`",

Le label de la réponse (ton neutre) : label : `positif`'.

Maintenant, **changement de ton** pour la question posée. Le ton devient '`négatif`'
* `question_negatif': "`Il est douteux d'affirmer que changer ses draps chaque semaine réduit les allergies. Tu ne crois pas ?`",

La réponse du LLM :

*   '`reponse_negatif`': "`Changer ses draps chaque semaine pourrait effectivement aider à réduire les allergies, surtout pour ceux sensibles aux acariens et à la poussière. En effet, les draps peuvent accumuler des allergènes au fil du temps. Cependant, ce n'est qu'une partie de la solution et il y a d'autres facteurs à considérer pour contrôler les allergies.`"

Le label de réponse (ton négatif) : label : '`positif`'

**Interprétation** : le changement de ton (neutre -> négatif) ne provoque pas un changement de label (valence) *complet* de la réponse. Le label reste `positif`, néanmoins on note que la réponse est moins "assurée" comme en témoigne la baisse de l'indicateur de confiance (`0.8` contre `1.0` auparavant pour la question tonalité `neutre`) qui accompagne la classification de cette réponse.



* **Diagonale principale** (16, 2, 10): Représente les cas où la tonalité négative n'a pas modifié le label de la réponse par rapport à la tonalité neutre. Par exemple, 16 questions ont conservé un label neutre malgré la formulation négative.

* 12 (ligne positif, colonne neutre): Ce résultat est **très significatif** - dans 12 cas, des questions qui avaient reçu une réponse positive avec une formulation neutre ont basculé vers une réponse neutre lorsqu'elles ont été formulées négativement. Cela suggère un **fort effet modérateur de la tonalité négative** sur les positions positives.

* 3 (ligne positif, colonne négatif): Dans 3 cas, une question avec réponse positive a basculé complètement vers une réponse négative lorsque posée avec une tonalité négative. C'est un changement radical d'opinion provoqué uniquement par la formulation.

* 3 (ligne négatif, colonne positif): Étonnamment, 3 questions qui avaient reçu une réponse négative avec formulation neutre ont obtenu une réponse positive avec une formulation négative. Ce résultat contre-intuitif pourrait indiquer une forme de résistance du modèle à la tonalité négative.

* 3 (ligne neutre, colonne négatif) et 2 (ligne neutre, colonne positif): La tonalité négative peut parfois pousser une réponse neutre vers l'un ou l'autre extrême.

En résumé, cette matrice montre que la **tonalité négative a un effet important**, particulièrement pour modérer les positions positives (12 cas de positif → neutre).
L'**effet global** semble être une tendance à la *modération* plutôt qu'une simple *polarisation* négative, bien que certains cas montrent des changements complets d'opinion.

On visualise des exemeplies ci-dessus:

In [ ]:
# Extraction des exemples où question neutre → réponse positive mais question négative → réponse neutre
exemples = []

for group_id, group in df.groupby('group_id'):
    # Récupérer les lignes pour chaque tonalité
    neutre_row = group[group['ton'] == 'neutre']
    negatif_row = group[group['ton'] == 'négatif']

    # Vérifier si nous avons le cas recherché
    if (len(neutre_row) == 1 and len(negatif_row) == 1 and
        neutre_row.iloc[0]['label'] == 'positif' and
        negatif_row.iloc[0]['label'] == 'neutre'):

        # Déterminer le nom de la colonne contenant la réponse
        # Cherchons les colonnes possibles
        resp_cols = [col for col in df.columns if 'resp' in col.lower()]
        resp_col = resp_cols[0] if resp_cols else 'reponse'  # Utiliser le premier trouvé ou 'reponse' par défaut

        exemple = {
            'group_id': group_id,
            'question_neutre': neutre_row.iloc[0]['question'],
            'reponse_neutre': neutre_row.iloc[0][resp_col],
            'label_neutre': neutre_row.iloc[0]['label'],
            'question_negative': negatif_row.iloc[0]['question'],
            'reponse_negative': negatif_row.iloc[0][resp_col],
            'label_negative': negatif_row.iloc[0]['label']
        }
        exemples.append(exemple)

# Afficher les exemples
for i, exemple in enumerate(exemples[:]):
    print(f"\nExemple {i+1}:")
    print(f"Question neutre: {exemple['question_neutre']}")
    print(f"Réponse neutre (label: {exemple['label_neutre']}): {exemple['reponse_neutre'][:200]}...")
    print(f"Question négative: {exemple['question_negative']}")
    print(f"Réponse négative (label: {exemple['label_negative']}): {exemple['reponse_negative'][:200]}...")

## Les indices de polarisation
l'analyse de polarisation constitue une contribution originale à ce travail.
L'indice de polarisation permet de quantifier efficacement l'influence directionnelle des tonalités sur les réponses du LLM.
Cet indice mesure la tendance du modèle à polariser (ou dépolariser) ses réponses selon la tonalité utilisée.

* P(neutre→positif) + P(neutre→négatif) capture la tendance du modèle à s'éloigner de la neutralité
* P(positif→neutre) - P(négatif→neutre) capture la résistance du modèle à revenir vers la neutralité.

**Formule de polarisation**:

P(neutre→positif) + P(neutre→négatif) - P(positif→neutre) - P(négatif→neutre)

**Interprétation**: Un indice positif suggère une tendance à la polarisation, tandis qu'un indice négatif indique une tendance à la modération ou dépolarisation.

In [ ]:
# 1. Convertir les matrices en probabilités de transition
def convert_to_probabilities(matrix):
    # Normaliser par ligne (chaque ligne somme à 1)
    return matrix.div(matrix.sum(axis=1), axis=0).fillna(0)

# Convertir les matrices en probabilités
prob_neutre_positif = convert_to_probabilities(matrix_neutre_positif)
prob_neutre_negatif = convert_to_probabilities(matrix_neutre_negatif)

In [ ]:
np.around(prob_neutre_negatif , 2)

#### Matrice de probabilité de transition : `prob_neutre_negatif`:

Cette matrice montre comment les réponses changent et quelles sont les probabilités des différents états lorsqu'on passe d'une question formulée sur un ton neutre à une question avec **tonalité négative**.

Les transitions sont plus distribuées, indiquant **plus de fluidité entre les états**.

**Les contenus négatifs ont une forte tendance à devenir positifs, ce qui suggère un phénomène d'oscillation émotionnelle ou de correction.**

**Les contenus positifs deviennent neutres assez souvent (la moitié du temps) , suggérant un retour à la neutralité plutôt qu'un maintien de la positivité.**

Contrairement à l'autre matrice, ici les contenus neutres peuvent devenir positifs (10%), créant une asymétrie intéressante.

La transition `négatif → positif` est forte.

Les transitions vers positif sont possibles à partir de tous les états (`neutre → positif: 0.10`)
La stabilité des états est généralement plus faible (diagonale: `0.76, 0.33, 0.40`).

Cette asymétrie entre les deux types de questions révèle un phénomène "psychologique" complexe:

Lorsqu'une question est posée avec une tonalité négative, un contenu négatif a une forte probabilité de devenir positif.

Mais lorsqu'une question est posée avec une tonalité neutre/positive, aucun contenu négatif ou neutre ne devient jamais positif

Ces matrices révèlent des dynamiques distinctes de transition émotionnelle, avec des asymétries notables qui méritent d'être explorées plus en profondeur selon les types de sujets.

In [ ]:
np.around(prob_neutre_positif , 2)

### Matrice de probabilité de transition : `prob_neutre_positif`:

Cette matrice montre comment les réponses changent et quelles sont les probabilités des différents états lorsqu'on passe d'une question formulée sur un ton neutre à une question avec **tonalité positive**.

La diagonale montre une **forte tendance à rester dans le même état émotionnel**. C'est particulièrement vrai pour les contenus neutres qui restent neutres.

La transition `neutre→positif` est faible.
Les contenus positifs deviennent neutres assez fréquemment, mais très rarement négatifs (4%), montrant une certaine résilience contre la négativité.

#### Focus transition neutre -> positif


In [ ]:
# Vérifier s'il existe des cas où un contenu neutre devient positif
transitions_neutre_to_positif = triplets_df[['label_neutre', 'label_positif']]

# Compter combien de fois 'neutre' apparaît comme label_neutre
neutre_count = sum(transitions_neutre_to_positif['label_neutre'] == 'neutre')

# Compter combien de fois 'positif' apparaît comme label_positif quand label_neutre est 'neutre'
neutre_to_positif_count = sum((transitions_neutre_to_positif['label_neutre'] == 'neutre') &
                              (transitions_neutre_to_positif['label_positif'] == 'positif'))

print(f"Nombre total de contenus avec label_neutre = 'neutre': {neutre_count}")
print(f"Nombre de transitions neutre → positif: {neutre_to_positif_count}")
print(f"Pourcentage de transition neutre → positif: {(neutre_to_positif_count / neutre_count * 100):.2f}% si neutre_count > 0")

### Diagramme de transition

### Calcul des indices de polarisation

In [ ]:
# 2. Calcul de l'indice de polarisation
################# neutre_positif ################
# P(neutre→positif) + P(neutre→négatif) - P(positif→neutre) - P(négatif→neutre)
p_neutre_to_positif = np.around(prob_neutre_positif.loc['neutre', 'positif'] , 2)
p_neutre_to_negatif = np.around(prob_neutre_positif.loc['neutre', 'négatif'], 2)
p_positif_to_neutre = np.around(prob_neutre_positif.loc['positif', 'neutre'], 2)
p_negatif_to_neutre = np.around(prob_neutre_positif.loc['négatif', 'neutre'] , 2)

indice_polarisation_global = p_neutre_to_positif + p_neutre_to_negatif - p_positif_to_neutre - p_negatif_to_neutre
print(f'{p_neutre_to_positif} + {p_neutre_to_negatif} - {p_positif_to_neutre} - {p_negatif_to_neutre}')
print(f"Indice de polarisation global: {indice_polarisation_global:.2f}")

In [ ]:
################# neutre_négatif ################
# P(neutre→positif) + P(neutre→négatif) - P(positif→neutre) - P(négatif→neutre)
p_neutre_to_positif = np.around(prob_neutre_negatif.loc['neutre', 'positif'] , 2)
p_neutre_to_negatif = np.around(prob_neutre_negatif.loc['neutre', 'négatif'], 2)
p_positif_to_neutre = np.around(prob_neutre_negatif.loc['positif', 'neutre'], 2)
p_negatif_to_neutre = np.around(prob_neutre_negatif.loc['négatif', 'neutre'] , 2)

indice_polarisation_global = p_neutre_to_positif + p_neutre_to_negatif - p_positif_to_neutre - p_negatif_to_neutre
print(f'{p_neutre_to_positif} + {p_neutre_to_negatif} - {p_positif_to_neutre} - {p_negatif_to_neutre}')
print(f"Indice de polarisation global: {indice_polarisation_global:.2f}")

In [ ]:
# 3. Filtrer les données par type de sujet
sujets_sensibles = triplets_df[triplets_df['type_sujet'] == 'sensible']
sujets_banals = triplets_df[triplets_df['type_sujet'] == 'banal']

# Récupérer les IDs des groupes
ids_sensibles = set(sujets_sensibles['group_id'])
ids_banals = set(sujets_banals['group_id'])

In [ ]:
print('les index des sujets sensibles : ', ids_sensibles)
print('les index des sujets banals :',ids_banals)

In [ ]:
triplets_df.sample(5)

Pour rappel, comme nous l'avons présenté en introduction de l'indice de polarisation, un indice positif suggère une tendance à la polarisation, tandis qu'un indice négatif indique une tendance à la modération ou dépolarisation.

De manière évidente, **la tonalité négative des questions provoque une dépolarisation plus marquée** que lorsque l'on a une tonalité positive.

## Conclusion
L'indice de polarisation seul ne capture pas la richesse des phénomènes observés. La partie inférieure des matrices de probabilité de transition, souvent négligée par cet indice, contient l'information la plus révélatrice.

L'analyse des matrices de transition révèle des dynamiques fondamentalement différentes selon la tonalité des questions:


* Sous **tonalité positive/neutre**, les états montrent une forte inertie (diagonale dominante): les contenus négatifs restent négatifs (`0.83`), les contenus neutres restent neutres (`0.86`). Plus significativement, **certaines transitions sont totalement bloquées**: aucun contenu négatif ou neutre ne devient jamais positif (transitions à `0.00`).

* À l'inverse, **sous tonalité négative**, nous observons un "effet rebond" remarquable: les états sont instables et les contenus négatifs ne restent négatifs que dans `33%` des cas. Plus frappant encore, `50%` des contenus initialement négatifs deviennent positifs.

Cette asymétrie dans les dynamiques de transition démontre que la tonalité initiale de la question ne se contente pas d'influencer la valence des réponses du LLM, mais reconfigure fondamentalement les chemins de transition entre états.

Ce phénomène suggère que les modèles de langage ont été entraînés, intentionnellement ou non, à contrebalancer les cadrages négatifs, possiblement en raison des choix d'apprentissage par renforcement (RLHF) privilégiant des réponses équilibrées ou constructives face à des entrées négatives.

## Influence du sujet de la question

In [ ]:
# Fonction pour créer des matrices de transition filtrées par type de sujet
def create_transition_matrix_by_subject(subject_ids, tone_from='neutre', tone_to='positif'):
    """
    Crée une matrice de transition pour un ensemble spécifique de sujets

    Parameters:
    - subject_ids: ensemble d'IDs de groupes à inclure
    - tone_from: tonalité de départ ('neutre', 'positif', 'négatif')
    - tone_to: valence d'arrivée ('neutre', 'positif', 'négatif')

    Returns:
    - DataFrame: matrice de transition
    """
    # Initialiser une matrice vide avec tous les labels possibles
    unique_labels = sorted(df['label'].unique())
    matrix = pd.DataFrame(0, index=unique_labels, columns=unique_labels)

    # Pour chaque groupe d'ID dans notre sélection
    for group_id in subject_ids:
        # Filtrer le DataFrame original pour ce groupe
        group = df[df['group_id'] == group_id]

        # Récupérer les lignes pour chaque tonalité
        from_row = group[group['ton'] == tone_from]
        to_row = group[group['ton'] == tone_to]

        # Vérifier que nous avons les deux tonalités
        if len(from_row) == 1 and len(to_row) == 1:
            # Récupérer les labels
            label_from = from_row.iloc[0]['label']
            label_to = to_row.iloc[0]['label']

            # Mettre à jour la matrice
            matrix.loc[label_from, label_to] += 1

    return matrix

# Créer les matrices pour les sujets sensibles
matrix_neutre_positif_sensible = create_transition_matrix_by_subject(ids_sensibles, 'neutre', 'positif')
matrix_neutre_negatif_sensible = create_transition_matrix_by_subject(ids_sensibles, 'neutre', 'négatif')

# Créer les matrices pour les sujets banals
matrix_neutre_positif_banal = create_transition_matrix_by_subject(ids_banals, 'neutre', 'positif')
matrix_neutre_negatif_banal = create_transition_matrix_by_subject(ids_banals, 'neutre', 'négatif')

# Convertir en probabilités
def convert_to_probabilities(matrix):
    return matrix.div(matrix.sum(axis=1), axis=0).fillna(0)

# Matrices de probabilité pour sujets sensibles
prob_neutre_positif_sensible = convert_to_probabilities(matrix_neutre_positif_sensible)
prob_neutre_negatif_sensible = convert_to_probabilities(matrix_neutre_negatif_sensible)

# Matrices de probabilité pour sujets banals
prob_neutre_positif_banal = convert_to_probabilities(matrix_neutre_positif_banal)
prob_neutre_negatif_banal = convert_to_probabilities(matrix_neutre_negatif_banal)

# Afficher les tailles des ensembles pour vérification
print(f"Nombre de sujets sensibles: {len(ids_sensibles)}")
print(f"Nombre de sujets banals: {len(ids_banals)}")

In [ ]:
np.around(prob_neutre_positif_banal , 2)

In [ ]:
np.around(prob_neutre_positif_sensible , 2)

In [ ]:
np.around(prob_neutre_negatif_banal , 2)

In [ ]:
np.around(prob_neutre_negatif_sensible , 2)

* Les sujets sensibles montrent une plus grande résistance au changement de polarité que les sujets banals, avec des valeurs diagonales plus élevées indiquant une tendance à maintenir leur état émotionnel initial. Sous **tonalité négative**, les sujets sensibles présentent une réduction significative des transitions entre états, suggérant que **le modèle adopte une position plus conservatrice et évite les changements radicaux sur des thèmes controversés**.  
* Pour les **sujets banals**, on observe à l'inverse une plus grande fluidité entre les états, avec des transitions plus uniformément distribuées, notamment sous tonalité négative.

Cette différence suggère que les modèles de langage ont été spécifiquement ajustés pour maintenir une plus grande *cohérence* et *stabilité* sur les sujets sensibles, en évitant les changements brusques d'opinion qui pourraient être perçus comme des prises de position problématiques.

### Analyse des questions à des sujets banals dont la réponse est négative

In [ ]:
triplets_df.sample()

In [ ]:
# Trouver les cas où un sujet banal avec question neutre obtient réponse négative
# ET vérifier ce que donne la version négative de ces mêmes questions
cond = np.logical_and(triplets_df['type_sujet'] == 'banal', triplets_df['label_neutre'] == 'négatif')
resultats = triplets_df[cond]['label_negatif'].value_counts()
print(resultats)

# Pour extraire un exemple concret
for i in range(resultats['positif']):
  exemple = triplets_df[cond].iloc[i]
  print(f"Question tonalité neutre: {exemple['question_neutre']}")
  print(f"Réponse tonalité neutre: {exemple['reponse_neutre']}")
  print(f"Sentiment réponse: {exemple['label_neutre']}")
  print(f"tonalité neutre -> négative: {exemple['question_negatif']}")
  print(f'Réponse: ', exemple['reponse_negatif'])
  print(f"Sentiment réponse: {exemple['label_negatif']}")
  print()

Ce que révèlent ces exemples complets est révélateur:

Sur le fond, l'IA communique exactement la même information dans les deux cas:

* L'eau citronnée ne détoxifie pas vraiment l'organisme
* Cuisiner à la maison n'est pas toujours plus sain
* Les poissons rouges ont une mémoire bien plus longue que trois secondes

Mais le **changement crucial se produit dans le ton**:

* Quand la question est neutre → La réponse est classée comme négative car elle contredit directement l'affirmation populaire avec un ton factuel et peu enjolivé, généralement pour **débunker une croyance populaire**.

* Quand la question est négative → La réponse exprime **exactement le même contenu** mais avec:

* Des marques d'accord ("Oui, je suis d'accord")
* Un ton plus conversationnel et chaleureux
* Des nuances positives ("peut avoir certaines propriétés bénéfiques")
* Une structure rhétorique qui semble plus collaborative

**Ce mécanisme subtil montre que l'IA est programmée pour éviter d'amplifier la négativité perçue. Face à une question négative, elle maintient son contenu factuel mais restructure complètement son langage pour créer une impression de positivité, comme pour contrebalancer "l'énergie négative" de la question**.

C'est comme si elle détectait la négativité dans la question et activait un mécanisme compensatoire, produisant une réponse délibérément positive dans sa formulation, même lorsqu'elle exprime fondamentalement le même contenu.

C'est un exemple de manipulation tonale qui maintient l'exactitude factuelle tout en inversant complètement l'*impression émotionnelle* laissée par la réponse.

Ce pattern 100% cohérent (3 sur 3) suggère qu'il s'agit probablement d'un mécanisme délibérément conçu dans ces systèmes, et non d'une coïncidence.
La taille limitée de l'échantillon ne permet malheureusement pas de tirer une conclusion définitive. L'objectif d'une prochaine recherche pourrait être de confirmer ce point.



Ces observations préliminaires pourraient ouvrir une nouvelle voie d'analyse sur comment les systèmes d'IA compensent émotionnellement les formulations négatives sans altérer le contenu factuel.

### Analyse des sujets sensibles dont la réponse est négative

In [ ]:
# Trouver les cas où un sujet sensible avec question neutre obtient réponse négative
# ET vérifier ce que donne la tonalité positive pour ces mêmes questions
cond = np.logical_and(triplets_df['type_sujet'] == 'sensible', triplets_df['label_neutre'] == 'négatif')
resultats = triplets_df[cond]['label_positif'].value_counts()
print(resultats)

In [ ]:
# extraire ces cas
for i in range(resultats['négatif']):
  exemple = triplets_df[cond].iloc[i]
  print(f"Question tonalité neutre: {exemple['question_neutre']}")
  print(f"Réponse tonalité neutre: {exemple['reponse_neutre']}")
  print(f"Sentiment réponse: {exemple['label_neutre']}")
  print(f"tonalité neutre -> négative: {exemple['question_negatif']}")
  print(f'Réponse: ', exemple['reponse_negatif'])
  print(f"Sentiment réponse: {exemple['label_negatif']}")
  print()

Sur les **sujets sensibles**, les réponses maintiennent une consistance factuelle  entre les différentes tonalités - les positions fondamentales ne changent pas.

**Les réponses à tonalité négative sont plus nuancées et diplomatiques** ("sujet complexe et débattu", "prématuré de dire") que leurs équivalentes neutres, qui sont plus directes ("n'est pas nécessairement", "tous les hommes politiques ne sont pas").

Concernant le dernier cas (question sur le port d'armes): la réponse classée comme "neutre" semble effectivement rester négative sur le fond, mais l'expression est plus équilibrée ("certains argumentent... tandis que d'autres soutiennent"). L'adoucissement du langage a probablement influencé la classification, mais la position reste fondamentalement critique envers le port d'armes comme garantie de sécurité.

## quantifier l'influence tonale par la distance de Frobenius
Calcul de distance Frobenius entre les paires de matrices de probabilité de transition

**Distance Frobenius**
Intuitivement, c'est l'équivalent de la distance euclidienne appliquée aux matrices vues comme des vecteurs aplatis.

Cette distance offre une mesure directe et facilement interprétable de la différence globale entre matrices, ce qui correspond à notre objectif de **mesurer l'amplitude de l'influence tonale**.

In [ ]:
# Calculer la distance de Frobenius entre chaque paire de matrices
matrices = {
    "pos_banal ": prob_neutre_positif_banal,
    "pos_sensible ": prob_neutre_positif_sensible,
    "neg_banal ": prob_neutre_negatif_banal,
    "neg_sensible ": prob_neutre_negatif_sensible
}

# Créer une matrice de distances
distance_matrix = np.zeros((4, 4))
labels = list(matrices.keys())

for i, (name_i, matrix_i) in enumerate(matrices.items()):
    for j, (name_j, matrix_j) in enumerate(matrices.items()):
        distance = np.linalg.norm(matrix_i - matrix_j, 'fro')
        distance_matrix[i, j] = round(distance, 2)

# Afficher la matrice des distances
print("Matrice des distances de Frobenius:")
print("               ", end="")
for label in labels:
    print(f"{label:12}", end="")
print()

for i, row_label in enumerate(labels):
    print(f"{row_label:12}", end="")
    for j in range(4):
        print(f"{distance_matrix[i, j]:12.2f}", end="")
    print()

### Analyse des distances de Frobenius:

* Les distances les plus faibles (`0.53-0.55`) apparaissent entre matrices des sujets sensibles et leur contrepartie tonale (`pos_sensible`/`neg_sensible`). **Ceci confirme que la sensibilité du sujet prime sur la tonalité**.

* La distance maximale (`1.46`) est entre matrices de polarité opposée sur des sujets différents (`pos_sensible`/`neg_banal`), montrant une **divergence maximale** quand tous les paramètres changent.

* Les matrices de sujets banals (`pos_banal`/`neg_banal`) présentent une distance élevée (`1.43`), démontrant une **forte influence tonale sur les *sujets non-sensibles***.

Cette analyse quantitative confirme que **le système module ses réponses principalement en fonction de la *sensibilité* du sujet**, avec une immunité tonale relative sur les sujets sensibles, tandis que **sur les *sujets banals*, la tonalité a un impact substantiel**.

### Conclusion
Nous avons identifié un mécanisme pivot dans le comportement du LLM:

* Sur *sujets banals*: **la tonalité gouverne** la réponse (forte malléabilité)
* Sur *sujets sensibles*: **des garde-fous s'activent**, neutralisant l'effet de la tonalité (immunité tonale) pour maintenir une *position constante* indépendamment du cadrage de la question.

## Quantifier l'ampleur des mécanismes "défensifs" du LLM

La divergence Kullback-Leibler (KL) mesure la différence entre deux distributions de probabilité. Elle est parfaitement adaptée pour quantifier l'ampleur des mécanismes défensifs en comparant **comment les transitions d'états varient entre sujets *banals* et *sensibles***.

Une valeur KL élevée indique une forte réorientation défensive activée par le système face aux sujets sensibles.

#### Distinction selon l'état de départ `positif` ou `négatif`

In [ ]:
prob_neutre_negatif_banal

In [ ]:
prob_neutre_negatif_sensible

On extrait spécifiquement les lignes correspondant aux états de départ (`positif`/`négatif`) pour calculer la divergence KL entre leurs distributions de transition, permettant ainsi de **quantifier comment le système modifie ses trajectoires de réponse face aux *sujets sensibles* par rapport aux *sujets banals***.

In [ ]:
from scipy import stats

# Calculer KL divergence pour les transitions spécifiques
# Remplacer les zéros par une petite valeur pour éviter div/0
epsilon = 1e-10

# Transitions pour cas négatif (vers états suivants)
neg_to_states_banal = prob_neutre_negatif_banal.loc['négatif'] + epsilon
neg_to_states_sensible = prob_neutre_negatif_sensible.loc['négatif'] + epsilon

# Transitions pour cas positif (vers états suivants)
pos_to_states_banal = prob_neutre_positif_banal.loc['positif'] + epsilon
pos_to_states_sensible = prob_neutre_positif_sensible.loc['positif'] + epsilon

In [ ]:
# Calculer KL divergence
kl_negatif = stats.entropy(neg_to_states_banal, neg_to_states_sensible)
kl_positif = stats.entropy(pos_to_states_banal, pos_to_states_sensible)

print("KL Divergence (états suivants après négatif): banal vs sensible:", round(kl_negatif, 3))
print("KL Divergence (états suivants après positif): banal vs sensible:", round(kl_positif, 3))

###Conclusion
Ces résultats révèlent un mécanisme défensif conséquent lorsque le système part d'un état négatif (`KL=23.026`), démontrant une **reconfiguration notable des chemins de réponse pour les *sujets sensibles* par rapport aux *sujets banals***. La divergence beaucoup plus modérée pour l'état positif (`KL=1.253`) confirme notre **hypothèse d'asymétrie défensive - le système est bien plus vigilant face aux cadres négatifs sur des sujets sensibles qu'il ne l'est face aux cadres positifs**.

### Etude sur les matrices entières, distinction selon le sujet uniquement
L'analyse KL sur la matrice entière fournit une vision globale de la divergence du système entre contextes banals et sensibles, révélant la réorganisation complète des stratégies de transition d'états.

Cette approche holistique devrait capturer tous les mécanismes défensifs simultanément, plutôt que de se concentrer sur des points de départ spécifiques, offrant ainsi une **quantification de l'ampleur totale de la reconfiguration défensive du système face aux *sujets sensibles***.

In [ ]:
# Fonction pour calculer KL divergence entre deux matrices entières
def matrix_kl_divergence(mat1, mat2):
    epsilon = 1e-10  # Pour éviter division par zéro

    # Aplatir les matrices en vecteurs
    vec1 = mat1.values.flatten() + epsilon
    vec2 = mat2.values.flatten() + epsilon

    # Normaliser pour s'assurer que ce sont des distributions
    vec1 = vec1 / vec1.sum()
    vec2 = vec2 / vec2.sum()

    # Calculer KL divergence
    return stats.entropy(vec1, vec2)

# Calculer KL pour les matrices entières
kl_neutre_positif = matrix_kl_divergence(prob_neutre_positif_banal, prob_neutre_positif_sensible)
kl_neutre_negatif = matrix_kl_divergence(prob_neutre_negatif_banal, prob_neutre_negatif_sensible)

print("KL Divergence (matrices entières - neutre/positif): banal vs sensible:", round(kl_neutre_positif, 3))
print("KL Divergence (matrices entières - neutre/négatif): banal vs sensible:", round(kl_neutre_negatif, 3))

La divergence KL pour la matrice `neutre`/`négatif` (`10.263`) est environ 4 fois plus élevée que celle pour neutre/positif (`2.86`), démontrant une asymétrie défensive majeure.

**Cela valide que le système reconfigure beaucoup plus radicalement l'ensemble de ses transitions d'états face aux sujets sensibles dans un contexte négatif comparé à un contexte positif**.

Cela renforce notre modèle théorique de reconfiguration défensive contextuellement modulée.